In [1]:
import numpy as np
import torch
from src.Data_fetching import get_pvgis_hourly
from src.Tensors import Tensors

import matplotlib.pyplot as plt

In [2]:
# Data fetching
latitude =52
longitude = 0
optimal_angles = 1                                                                    # 1 if azimuth and tilt are not known, otherwise 0
azimuth = None                                                                        # None if azimuth is not known
tilt = None                                                                           # None if tilt is not known
peak_power = 2.48                                                                     # The peak output of the system
loss = 14                        

In [3]:
start_date = 2005
end_date = None

parameters_PVGIS = {'lat': latitude,
              'lon': longitude,
              'startyear': start_date,
              'endyear': end_date,
              'outputformat': 'json',
              'angle': tilt,
              'aspect': azimuth,
              'optimalangles': optimal_angles,
              'pvcalculation': 1,
              'components': 1,
              'peakpower': peak_power,
              'loss': loss,
              'localtime': 0}

In [4]:
pv_data = get_pvgis_hourly(parameters_PVGIS)[['P']]

In [5]:
pv_data.index = pv_data.index.floor('h').tz_localize(None)

In [6]:
pv_data.loc[:, 'hour_sin'] = np.sin(2 * np.pi * pv_data.index.hour / 24)
pv_data.loc[:, 'hour_cos'] = np.cos(2 * np.pi * pv_data.index.hour / 24)

In [7]:
pv_data

,P,hour_sin,hour_cos
time,,,
2005-01-01 00:00:00,0.0,0.000000,1.000000
2005-01-01 01:00:00,0.0,0.258819,0.965926
2005-01-01 02:00:00,0.0,0.500000,0.866025
2005-01-01 03:00:00,0.0,0.707107,0.707107
2005-01-01 04:00:00,0.0,0.866025,0.500000
...,...,...,...
2020-12-31 19:00:00,0.0,-0.965926,0.258819
2020-12-31 20:00:00,0.0,-0.866025,0.500000
2020-12-31 21:00:00,0.0,-0.707107,0.707107


In [8]:
past_features = ['P']                                                                 
future_features = ['hour_sin',                                                        
                   'hour_cos']

lags = 24                                                                             
forecast_period = 24                                                                  
gap = -23
forecast_gap = -23

In [9]:
# Get the data in the torch.tensor format
target_tensors = Tensors(pv_data,
                         'P',
                         past_features ,
                         future_features,
                         lags,
                         forecast_period,
                         gap=gap,
                         forecast_gap=forecast_gap,
                         train_test_split=0.8)

# Split the data into train and test sets with separate tensors for features (X) and the target (y)
X_train, X_test, y_train, y_test = target_tensors.create_tensor()
X_cal = X_train[-X_test.shape[0]:,:,:]
X_train = X_train[:-X_test.shape[0],:,:]
y_cal = y_train[-y_test.shape[0]:,:]
y_train = y_train[:-y_test.shape[0],:]

X_train.shape, X_cal.shape, X_test.shape, y_train.shape, y_cal.shape, y_test.shape

(torch.Size([84140, 24, 3]),
 torch.Size([28046, 24, 3]),
 torch.Size([28046, 24, 3]),
 torch.Size([84140, 24]),
 torch.Size([28046, 24]),
 torch.Size([28046, 24]))

In [10]:
from src.DeepAR import DeepAR
from src.DeepAR import Training

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
deepar = DeepAR(input_size=3,
                hidden_size=400,
                num_layers=4,
                dropout=0.5,
                )

In [13]:
trainer = Training(deepar,X_train,y_train, X_test, y_test, epochs=100,batch_size=64, learning_rate=0.001)

In [14]:
trainer.train()

Epoch 1/100, Loss: -1.5880
Epoch 2/100, Loss: -2.0020
Epoch 3/100, Loss: -2.1103
Epoch 4/100, Loss: -2.4564
Epoch 5/100, Loss: -2.5315
Epoch 6/100, Loss: -2.3368
Epoch 7/100, Loss: -2.6363
Epoch 8/100, Loss: -2.4959
Epoch 9/100, Loss: -2.4234
Epoch 10/100, Loss: -2.8084
Epoch 11/100, Loss: -2.2978
Epoch 12/100, Loss: -2.7672
Epoch 13/100, Loss: -2.4097
Epoch 14/100, Loss: -2.7154
Epoch 15/100, Loss: -1.9090
Epoch 16/100, Loss: -2.7118
Epoch 17/100, Loss: -2.1503
Epoch 18/100, Loss: -0.7861
Epoch 19/100, Loss: -2.5494
Epoch 20/100, Loss: -2.3120
Epoch 21/100, Loss: -2.3642
Epoch 22/100, Loss: -2.3971
Epoch 23/100, Loss: -2.6500
Epoch 24/100, Loss: -2.2148
Epoch 25/100, Loss: -2.6569
Epoch 26/100, Loss: -2.5588
Epoch 27/100, Loss: -2.6730
Epoch 28/100, Loss: -2.5320
Epoch 29/100, Loss: -2.5363
Epoch 30/100, Loss: -2.5932
Epoch 31/100, Loss: -2.6168
Epoch 32/100, Loss: -2.6810
Epoch 33/100, Loss: -2.6778
Epoch 34/100, Loss: -1.7284
Epoch 35/100, Loss: -2.4842
Epoch 36/100, Loss: -2.0755
E

In [17]:
mu, sigma = deepar(X_test[0,:,:].to(device))